In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Checking Total Derivatives

If you want to check the analytic derivatives of your model (or just part of it) against finite difference or complex-step approximations, you can use `check_totals`(). You should always converge your model before calling this method. By default, this method checks the derivatives of all of the driver responses (objectives, constraints) with respect to the design variables, though you can also specify the variables you want to check. Derivatives are computed and compared in an unscaled form by default, but you can optionally request for them to be computed in scaled form using the `ref` and `ref0` that were declared when adding the constraints, objectives, and design variables.

```{Note}
You should probably **not** use this method until you’ve used `check_partials`() to verify the partials for each component in your model. `check_totals`() is a blunt instrument, since it can only tell you that there is a problem, but will not give you much insight into which component or group is causing the problem.

``` 

```{eval-rst}
    .. automethod:: openmdao.core.problem.Problem.check_totals
        :noindex:
```

Please check out the [Complex Step Guidelines](complex-step-guidelines) for more information on how to use it on a model.

## Examples

You can check specific combinations of variables by specifying them manually:

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src63", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src63`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.setup()
prob.run_model()

In [ ]:
# manually specify which derivatives to check
prob.check_totals(of=['obj', 'con1'], wrt=['x', 'z'])

---
Check all the derivatives that the driver will need:

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

In [ ]:
# check derivatives of all obj+constraints w.r.t all design variables
prob.check_totals()

---
Use the driver scaled values during the check:

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100, ref=100.0, ref0=-100.0)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0, ref=3.0)
prob.model.add_constraint('con2', upper=0.0, ref=20.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

In [ ]:
# check derivatives of all driver vars using the declared scaling
prob.check_totals(driver_scaling=True)


---
Display the results in a compact format:

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

In [ ]:
# check derivatives of all obj+constraints w.r.t all design variables
prob.check_totals(compact_print=True)

---
Use complex step instead of finite difference for a more accurate check. We also tighten the tolerance on the nonlinear Gauss-Seidel solver so that we get more accurate converged values.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup(force_alloc_complex=True)

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

In [ ]:
prob.model.nonlinear_solver.options['atol'] = 1e-15
prob.model.nonlinear_solver.options['rtol'] = 1e-15

# check derivatives with complex step
prob.check_totals(method='cs')

Note that you need to set the argument `force_alloc_complex` to True when you call `setup`. This makes sure that the vectors allocate space to pass complex numbers. All systems also have an attribute `force_alloc_complex` that can be queried during `setup` in case you need to set up your `Component` or `Group` differently to accomodate complex step.

In [ ]:
import openmdao.api as om

class SimpleComp(om.ExplicitComponent):

    def setup(self):
        self.add_input('x', val=1.0)
        self.add_output('y', val=1.0)

        self.declare_partials(of='y', wrt='x')

        if self._force_alloc_complex:
            print("Vectors allocated for complex step.")

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0*inputs['x']

    def compute_partials(self, inputs, partials):
        partials['y', 'x'] = 3.

prob = om.Problem()
prob.model.add_subsystem('comp', SimpleComp())

prob.model.add_design_var('comp.x', lower=-100, upper=100)
prob.model.add_objective('comp.y')

prob.setup(force_alloc_complex=True)

prob.run_model()

prob.check_totals(method='cs')


---
Turn off standard output and just view the derivatives in the return:

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

In [ ]:
# check derivatives of all obj+constraints w.r.t all design variables
totals = prob.check_totals(out_stream=None)

In [ ]:
print(totals)


---
Evaluate the total derivatives using multiple FD step sizes:

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

In [ ]:
# check derivatives of all obj+constraints w.r.t all design variables for FD steps of 1e-6 and 1e-7
prob.check_totals(step=[1.e7, 1.e8])